# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats

In [2]:
df = pd.read_csv("homepage_actions.csv")
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [3]:
df["group"].value_counts()

control       4264
experiment    3924
Name: group, dtype: int64

In [4]:
df["action"].value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [5]:
duplicated = df[df.duplicated("id")]
duplicated

,timestamp,id,group,action
8,2016-09-24 20:58:01.948663,349125,experiment,click
12,2016-09-24 21:06:27.553057,601714,experiment,click
15,2016-09-24 21:30:02.739756,487634,experiment,click
17,2016-09-24 23:01:12.108316,468601,experiment,click
21,2016-09-25 00:01:47.933853,555973,experiment,click
...,...,...,...,...
8162,2017-01-17 23:20:35.483601,451198,control,click
8164,2017-01-17 23:47:58.209653,252195,control,click
8167,2017-01-18 00:56:24.554729,344770,experiment,click
8180,2017-01-18 08:54:56.879682,615849,experiment,click


In [6]:
control_duplicated = duplicated.loc[duplicated['group'] == 'control']
control_duplicated

,timestamp,id,group,action
28,2016-09-25 02:53:25.459874,398892,control,click
37,2016-09-25 05:19:15.810727,544571,control,click
56,2016-09-25 08:25:32.821891,194950,control,click
58,2016-09-25 09:45:12.114972,894454,control,click
63,2016-09-25 10:38:53.299877,639852,control,click
...,...,...,...,...
8149,2017-01-17 18:32:30.832981,762498,control,click
8160,2017-01-17 22:40:54.304413,591686,control,click
8162,2017-01-17 23:20:35.483601,451198,control,click
8164,2017-01-17 23:47:58.209653,252195,control,click


In [7]:
experiment_duplicated = duplicated.loc[duplicated['group'] == 'experiment']
experiment_duplicated

,timestamp,id,group,action
8,2016-09-24 20:58:01.948663,349125,experiment,click
12,2016-09-24 21:06:27.553057,601714,experiment,click
15,2016-09-24 21:30:02.739756,487634,experiment,click
17,2016-09-24 23:01:12.108316,468601,experiment,click
21,2016-09-25 00:01:47.933853,555973,experiment,click
...,...,...,...,...
8117,2017-01-17 09:27:37.808307,891635,experiment,click
8133,2017-01-17 15:09:17.533736,865656,experiment,click
8152,2017-01-17 20:14:12.483794,413359,experiment,click
8167,2017-01-18 00:56:24.554729,344770,experiment,click


In [8]:
"""
There are 8188 people.
1860 people in total clicked and viewed.
932 people in control clicked and viewed,
while 928 people in the experiment clicked and viewed.
"""

'\nThere are 8188 people.\n1860 people in total clicked and viewed.\n932 people in control clicked and viewed,\nwhile 928 people in the experiment clicked and viewed.\n'

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [9]:
"""
NULL: There is no difference in effectiveness of the control and experimental website.
ALTERNATIVE: The experimental site is more effective in interacting with the user.
"""

'\nNULL: There is no difference in effectiveness of the control and experimental website.\nALTERNATIVE: The experimental site is more effective in interacting with the user.\n'

In [10]:
len(df.loc[df['group'] == 'control']) - len(control_duplicated)

3332

In [11]:
len(df.loc[df['group'] == 'experiment']) - len(experiment_duplicated)

2996

In [12]:
"""
3332 samples in control group
2996 samples in experimental group
"""

'\n3332 samples in control group\n2996 samples in experimental group\n'

In [13]:
control_click_rate = 932/3332
control_click_rate

0.2797118847539016

In [14]:
experiment_click_rate = 928/2996
experiment_click_rate

0.3097463284379172

In [15]:
df['count'] = 1

In [16]:
control = df.loc[df['group'] == 'control'].pivot(index='id', columns='action', values='count')
control = control.fillna(0)
control

action,click,view
id,,
182994,1.0,1.0
183089,0.0,1.0
183248,1.0,1.0
183515,0.0,1.0
183524,0.0,1.0
...,...,...
936786,0.0,1.0
937003,0.0,1.0
937073,0.0,1.0


In [17]:
experiment = df.loc[df['group'] == 'experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(0)
experiment

action,click,view
id,,
182988,0.0,1.0
183136,0.0,1.0
183141,1.0,1.0
183283,0.0,1.0
183389,0.0,1.0
...,...,...
935382,0.0,1.0
935576,0.0,1.0
935742,1.0,1.0


In [18]:
ttest = stats.ttest_ind(control.click, experiment.click, equal_var = False)
ttest.pvalue/2

0.004466402814337102

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [19]:
expected_experimental = control_click_rate * len(experiment['view'])
expected_experimental

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [20]:
n = len(experiment['view'])
p = control_click_rate

In [21]:
var = n * p * (1 - p)
std = np.sqrt(var)
std

24.568547907005815

In [22]:
actual_experimental = 928
z_score = (actual_experimental - expected_experimental)/std
z_score

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [23]:
import scipy.stats as stats
p_val = stats.norm.sf(z_score)
p_val

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

In [24]:
"""
The p-values are not exactly the same but both values are below 0.05.
Therefore we can confidently reject the null hypothesis.
"""

'\nThe p-values are not exactly the same but both values are below 0.05.\nTherefore we can confidently reject the null hypothesis.\n'

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.